<a href="https://colab.research.google.com/github/Atria14/data-science/blob/main/06_ML/02_metryki_klasyfikacja.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### scikit-learn
>Strona biblioteki: [https://scikit-learn.org](https://scikit-learn.org)  
>
>Dokumentacja/User Guide: [https://scikit-learn.org/stable/user_guide.html](https://scikit-learn.org/stable/user_guide.html)
>
>Podstawowa biblioteka do uczenia maszynowego w języku Python.
>
>Aby zainstalować bibliotekę scikit-learn, użyj polecenia poniżej:
```
pip install scikit-learn
```
### Metryki - Problem klasyfikacji:
1. [Import bibliotek](#a0)
2. [Metryki - Klasyfikacja binarna](#a1)
    1. [Accuracy - dokładność klasyfikacji](#a2)
    2. [Macierz konfuzji/pomyłek](#a3)
    3. [Krzywa ROC](#a4)
3. [Metryki - Klasyfikacja wieloklasowa](#a5)   
    1. [Accuracy - dokładność klasyfikacji](#a6)
    2. [Macierz konfuzji/pomyłek](#a7)
    3. [Raport klasyfikacji](#a8)

#### <a name='a0'></a> Import bibliotek

In [1]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

#### <a name='a1'></a>  Metryki - Klasyfikacja binarna
#### <a name='a2'></a> Accuracy - Dokładność klasyfikacji

####   $$Accuracy = \frac{correct\ predictions}{total\ predictions} * 100$$

In [2]:
y_true = np.array([1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1])
y_pred = np.array([0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1])

In [3]:
from sklearn.metrics import accuracy_score # funkcja do określenia dokładności klasyfikacji

accuracy_score(y_true, y_pred) # pierwsza wartość - etykiety prawdziwe, druga - etykiety które przewidział model (tutaj są podane ręcznie, przy normalnej pracy będą wygenerowane z modelu ML)

0.8

In [4]:
# dla tego zestawu danych dokładność to 80%
# tworzymy obiekt DataFrame z naszymi danymi true i pred w formie słownika
results = pd.DataFrame({'y_true': y_true, 'y_pred': y_pred})
results
# chcemy wykorzystać plotly - dlatego przekształciliśmy obiekt na dataframe

,y_true,y_pred
0,1,0
1,0,0
2,1,1
3,1,1
4,0,0
5,1,1
6,1,0
7,0,0
8,1,1
9,0,1


In [5]:
# 1. sortujemy dane wg y_true funkcją sort_values(by=zmienna po jakiej chcemy sortować), przypisujemy do zmiennej results aby zmiany były zachowane w zmiennej
results = results.sort_values(by='y_true')
# 2. dodajemy kolumnę, resetujemy indeks funkcją reset_index
results = results.reset_index(drop=True)
# 3. tworzymy nową kolumnę sample z indexsem powiększonym o 1
results['sample'] = results.index + 1
results

,y_true,y_pred,sample
0,0,0,1
1,0,0,2
2,0,0,3
3,0,0,4
4,0,0,5
5,0,0,6
6,0,0,7
7,0,0,8
8,0,1,9
9,0,1,10


In [6]:
# tworzymy obiekt fig, podajemy rows - 2 wiersze, cols 1 - jedna kolumna
fig = make_subplots(rows=2, cols=1)
# podajemy typ wykresu - tutaj Scatter, dane dla osi x i y, bastępnie nazwa i wpisujemy w którą komórkę wykres ma zostać włożony row 1, col 1
fig.add_trace(go.Scatter(x=results['sample'], y=results['y_true'], mode='markers', name='y_true'), row=1, col=1)
# taki sam wykres dla predykcji:
fig.add_trace(go.Scatter(x=results['sample'], y=results['y_pred'], mode='markers', name='y_pred'), row=2, col=1)
# aktualizujemy layout
fig.update_layout(width=900, height=500, title='Klasyfikator binarny')
fig.show()

# im wykresy są bardziej podobne do siebie tym lepiej radzi sobie nasz model

#### <a name='a3'></a> Macierz konfuzji/pomyłek

In [7]:
#importujemy confusion_matrix - macierz konfuzji
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_true, y_pred)
# ponieważ nasz problem jest dwyklasowy (binarny) dostajemy macierz 2 x 2
# interesuje nas maksymalizacja wartości na przekątnych
# ważne aby przypisać macierz konfuzji do zmiennej (tutaj cm), której użyjemy do wyświetlenia wykresu

In [8]:
# tworzymy wykres który wyświetli nam interaktywną macierz konfuzji

import plotly.figure_factory as ff

def plot_confusion_matrix(cm):
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1'], index=['true_1', 'true_0'])

    fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index), 
                                      colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(width=400, height=400, title='Confusion Matrix', font_size=16)
    fig.show()

plot_confusion_matrix(cm)

In [9]:
cm.ravel()

array([ 8,  2,  4, 16])

In [10]:
# wynaczanie metryk na podstawie macierzy konfuzji
tn, fp, fn, tp = cm.ravel()
print(f'TN - True Negative: {tn}')
print(f'FP - False Positive: {fp}')
print(f'FN - False Negative: {fn}')
print(f'TP - True Positive: {tp}')

TN - True Negative: 8
FP - False Positive: 2
FN - False Negative: 4
TP - True Positive: 16


In [11]:
# Type I error - False Positive Rate, błąd pierwszego rodzaju
# wartość błędu pierwszego rodzaju:
fpr = fp / (fp + tn)
fpr
# wskaźnik predykcji fałszywie pozytywnych

0.2

In [12]:
# Type II error - False Negative Rate
# błąd drugiego rodzaju
fnr = fn / (fn + tp)
fnr
# wskaźnik predykcji fałszywie negatywnych

0.2

In [13]:
# Precision - ile obserwacji przewidywanych jako pozytywne są w rzeczywistości pozytywne
precision = tp / (tp + fp)
print(precision)

0.8888888888888888


In [14]:
# Recall - jak wiele obserwacji z wzystkich poytywnych sklasyfikowaliśmy jako pozytywne
recall = tp / (tp + fn)
print(recall)

0.8


#### <a name='a4'></a> Krzywa ROC

In [15]:
from sklearn.metrics import roc_curve

fpr, tpr, thresh = roc_curve(y_true, y_pred, pos_label=1) # fpr - false positive rate, tpr - true positive rate, threshold,pos_label - klasa pozytywna u nas 1
roc = pd.DataFrame({'fpr': fpr, 'tpr': tpr})
roc

,fpr,tpr
0,0.0,0.0
1,0.2,0.8
2,1.0,1.0


In [16]:
def plot_roc_curve(y_true, y_pred):
    # Binary classification
    from sklearn.metrics import roc_curve
    fpr, tpr, tresh = roc_curve(y_true, y_pred, pos_label=1)

    fig = go.Figure(data=[go.Scatter(x=roc['fpr'], y=roc['tpr'], line_color='red', name='ROC Curve'),
                        go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line_dash='dash', line_color='navy')],
                    layout=go.Layout(xaxis_title='False Positive Rate',
                                    yaxis_title='True Positive Rate',
                                    title='ROC Curve',
                                    showlegend=False,
                                    width=800,
                                    height=400))
    fig.show()
plot_roc_curve(y_true, y_pred)

# w przypadku analizy więcej niż jednego modelu, na krzywej ROC będzie więcej linii. Najlepszy jest ten model, który ma największe pole pod wykresem. Częśto oblicza się pole pod wykresem w celu doboru najlepszego modelu klasifikiacji binarnej.
# linia przerywana - to model losowy

#### <a name='a5'></a> Metryki - Klasyfikacja wieloklasowa

#### <a name='a6'></a> Accuracy - Dokładność klasyfikacji

####   $$Accuracy = \frac{correct\ predictions}{total\ predictions} * 100$$

In [17]:
# 3 klasy, true - prawdziwe, pred - przewidziane na podstawie modelu, niektóre klasy są źle przewidziane dlatego będziemy szukać rozwiązania aby sprawdzić dokładność modelu
y_true = np.array([1, 0, 1, 2, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 2, 1, 1, 2, 2, 1, 0, 1, 1, 0, 2, 1, 1, 2, 2])
y_pred = np.array([0, 0, 1, 2, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 2, 1, 2, 1, 2, 1, 0, 2, 1, 0, 1, 1, 1, 2, 2])

accuracy_score(y_true, y_pred)

0.7241379310344828

#### <a name='a7'></a> Macierz konfuzji/pomyłek

In [18]:
cm = confusion_matrix(y_true, y_pred) # mamy 3 klasy, macierz 3x3, przy 10 klasach było by 10x10
cm

array([[ 6,  1,  0],
       [ 3, 10,  2],
       [ 0,  2,  5]])

In [19]:
def plot_confusion_matrix(cm):
    cm = cm[::-1]
    cm = pd.DataFrame(cm, columns=['pred_0', 'pred_1', 'pred_2'], index=['true_2','true_1', 'true_0'])

    fig = ff.create_annotated_heatmap(z=cm.values, x=list(cm.columns), y=list(cm.index), 
                                      colorscale='ice', showscale=True, reversescale=True)
    fig.update_layout(width=400, height=400, title='Confusion Matrix', font_size=16)
    fig.show()

plot_confusion_matrix(cm)

# poniżej raport klasyfikacji
# dla wartości 0 - model 6x przewidział wynik poprawny, dla wartości 1 - 1x model przewidział wartość błędnie
# dla wartości 1 - model 10x przewidział poprawnie, 3x przewidział błędnie wartość 0 a 2x będnie wartość 2
# dla wartości 2 - model 5x przewidział poprawnie wartość 2, 2x przewidział błędnie wartość 1

#### <a name='a8'></a> Raport klasyfikacji

In [20]:
from sklearn.metrics import classification_report

print(classification_report(y_true, y_pred))
# f1-score to średnia harmoniczna wartości precision i recall

              precision    recall  f1-score   support

           0       0.67      0.86      0.75         7
           1       0.77      0.67      0.71        15
           2       0.71      0.71      0.71         7

    accuracy                           0.72        29
   macro avg       0.72      0.75      0.73        29
weighted avg       0.73      0.72      0.72        29



In [21]:
print(classification_report(y_true, y_pred, target_names=['label_1', 'label_2', 'label_3'])) # nadajemy etykiety (nazwy) klasom

              precision    recall  f1-score   support

     label_1       0.67      0.86      0.75         7
     label_2       0.77      0.67      0.71        15
     label_3       0.71      0.71      0.71         7

    accuracy                           0.72        29
   macro avg       0.72      0.75      0.73        29
weighted avg       0.73      0.72      0.72        29

